In [1]:
##This Iteration calculates the QoQ revenue change as the y_tag
import dropbox
import urllib.request, json 
from collections import defaultdict
from collections import OrderedDict
import datetime as DT
import ast
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
import scipy.sparse
from string import punctuation
import csv
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

# Helper libraries
from w266_common import utils, vocabulary

# Import pandas and tensorflow
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.10"))

# Import model
import models

# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Import misc
from importlib import reload
import os, sys, re, json, time, datetime, shutil
import itertools
from w266_common import patched_numpy_io

/home/danlee85/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##Download the pre-processed tweets from dropbox

access_token = 'p3Ze8FpdRTAAAAAAAAAADb9hCQ8aXXYU3A3gGM1HEXyeMrs8KI2SrA71KDmmCXte'
dbx = dropbox.Dropbox(access_token)



access_token2 = 'm_fKi8mWZ0AAAAAAAAAADlfzTm37f1y45k92Xpiw1B6mjT3dQqzNvKgpIUSR17uq'
dbx2 = dropbox.Dropbox(access_token2)

# #Write the combined file into the drive
with open("InputX.txt", "wb") as f:
    metadata, res = dbx.files_download(path="/Inputdata2.txt")
    metadata2, res2 = dbx2.files_download(path="/Inputdata.txt")
    f.write(res.content)
    f.write(res2.content)

In [2]:
##Translate date of the tweet to quarter reference
def Quarterlookup(date):
    if date>=DT.datetime(2018,10,1):
        return '18Q4'
    elif date>=DT.datetime(2018,7,1):
        return '18Q3'
    elif date>=DT.datetime(2018,4,1):
        return '18Q2'
    elif date>=DT.datetime(2018,1,1):
        return '18Q1'
    elif date>=DT.datetime(2017,10,1):
        return '17Q4'
    elif date>=DT.datetime(2017,7,1):
        return '17Q3'
    elif date>=DT.datetime(2017,4,1):
        return '17Q2'
    elif date>=DT.datetime(2017,1,1):
        return '17Q1'
    elif date>=DT.datetime(2016,10,1):
        return '16Q4'
    elif date>=DT.datetime(2016,7,1):
        return '16Q3'
    elif date>=DT.datetime(2016,4,1):
        return '16Q2'
    elif date>=DT.datetime(2016,1,1):
        return '16Q1'
    elif date>=DT.datetime(2015,10,1):
        return '15Q4'
    elif date>=DT.datetime(2015,7,1):
        return '15Q3'
    elif date>=DT.datetime(2015,4,1):
        return '15Q2'
    else:
        return '15Q1'

In [3]:
##Reading the Input X file and process it
d=defaultdict(list)

with open('InputX.txt') as f:
    for line in f:
        count=0
        temp =-1
        k,v=line.split(':',1)
        while True:
            
            oldtemp=temp
            temp=v.find(']]',temp+1)
            if count==0:
                d[k].append(v[oldtemp+2:temp+2]) 
            else:
                d[k].append(v[oldtemp+4:temp+2])
           
             
            count+=1
            if temp == -1: break
            #if count == 500: break  # initially limit for time sake
    

In [4]:
#Read from the pre prepared vocab file
allword=set()
with open('vocab1.csv', 'r') as f:
    for line in f:
        allword.add(line.strip())

In [5]:
allword=sorted(allword)
vocab = vocabulary.Vocabulary(allword, size=None)  
print("Vocabulary size: {:,}".format(vocab.size))

Vocabulary size: 5,855


In [6]:
#Create the padding
def pad_np_array(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    cpy_len = min(len(example_ids), max_len)

    arr = example_ids[:cpy_len]
    ns = cpy_len
    return arr, ns

In [7]:
#Dictionary to collect the tweet detail and the length of each tweet
InputX=defaultdict(dict)
InputN=defaultdict(dict)

In [8]:
#Set of all the stocks of interest
trainstock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']
# teststock =['CVSHealth', 'RiteAid', "Conn's", "Casey's", 'Kroger','WeisMarkets', 'HavertyFurniture', "Kirkland's", 'Pier1Imports', 'BuildersFirstSource', 'HuttigBuildingProducts', 'LumberLiquidators', 'TransWorldEntertainment', 'MarineMax', 'Medifast', 'SallyBeauty',"Sotheby's", 'StarGasPartners', 'SuburbanPropanePartners', 'TitanMachinery', 'TractorSupply', 'TravelCentersofAmerica', 'Big5SportingGoods', "Dick'sSportingGoods", 'HibbettSports', 'Build-A-BearWorkshop']

# trainstock =["TitanMachinery"]
teststock =trainstock


In [9]:
##Converting the string of data into dictionary


for key,value in d.items():
    Inputx=defaultdict(list)
    Inputn=defaultdict(list)
    if key in trainstock:
        for value2 in value:
        
            temp=value2.find('), ')
            date=value2[14:temp+3]
            try:
                year=int(date[1:5])
                month=int(date[date.find(',')+2:date.find(',',date.find(',')+1)])
                day=int(date[date.find(')')-2:date.find(')')])
            
                sentence, length = pad_np_array (vocab.words_to_ids(ast.literal_eval(value2[temp+3:-1])))
                ##Storing in content of each tweet and length of each tweet in sub level variable
                Inputx[Quarterlookup(DT.datetime(year,month,day))].append(sentence)
                Inputn[Quarterlookup(DT.datetime(year,month,day))].append(length)
            except:
                pass

    ##Storing in content of each tweet and length of each tweet in the top level variable
    InputX[key]=OrderedDict(sorted(Inputx.items()))
    InputN[key]=OrderedDict(sorted(Inputn.items()))

In [10]:
##List of stockname and relevant stock code
l = [("CVSHealth", "CVS"),
("RiteAid", "RAD"),
("Conn's", "CONN"),
("Casey's", "CASY"),
("Kroger", "KR"),
("WeisMarkets", "WMK"),
("HavertyFurniture", "HVT"),
("Kirkland's", "KIRK"),
("Pier1Imports","PIR"),
("BuildersFirstSource","BLDR"),
("HuttigBuildingProducts", "HBP"),
("LumberLiquidators", "LL"),
("TransWorldEntertainment", "TWMC"),
("MarineMax", "HZO"),
("Medifast", "MED"),
("SallyBeauty", "SBH"),
("Sotheby's", "BID"),
("StarGasPartners", "SGU"),
("SuburbanPropanePartners", "SPH"),
("TitanMachinery", "TITN"),
("TractorSupply","TSCO"),
("TravelCentersofAmerica", "TA"),
("Big5SportingGoods", "BGFV"),
("Dick'sSportingGoods", "DKS"),
("HibbettSports", "HIBB"),
("Build-A-BearWorkshop", "BBW")]
     
all_stock = {}
[all_stock.update({k:v}) for k,v in l]
print(all_stock)

{'CVSHealth': 'CVS', 'RiteAid': 'RAD', "Conn's": 'CONN', "Casey's": 'CASY', 'Kroger': 'KR', 'WeisMarkets': 'WMK', 'HavertyFurniture': 'HVT', "Kirkland's": 'KIRK', 'Pier1Imports': 'PIR', 'BuildersFirstSource': 'BLDR', 'HuttigBuildingProducts': 'HBP', 'LumberLiquidators': 'LL', 'TransWorldEntertainment': 'TWMC', 'MarineMax': 'HZO', 'Medifast': 'MED', 'SallyBeauty': 'SBH', "Sotheby's": 'BID', 'StarGasPartners': 'SGU', 'SuburbanPropanePartners': 'SPH', 'TitanMachinery': 'TITN', 'TractorSupply': 'TSCO', 'TravelCentersofAmerica': 'TA', 'Big5SportingGoods': 'BGFV', "Dick'sSportingGoods": 'DKS', 'HibbettSports': 'HIBB', 'Build-A-BearWorkshop': 'BBW'}


In [11]:
##Read the quarterly return detail from edgaronline and convert into library
top="http://datafied.api.edgar-online.com/v2/corefinancials/qtr?primarysymbols="
bottom="&appkey=asc97xrhkyu4959aptu76zxj"
Outputy =defaultdict(dict)
for stockname,stock in all_stock.items():
    check=top+stock+bottom
    with urllib.request.urlopen(check) as url:
        data = json.loads(url.read().decode())
    count1=0
    count2=0
    end_date=[]
    revenue_change=[]
    last_rev=0

    for key, value in data.items():
        
        for key2, value2 in value.items():          
        
            if count1!=0:  
                for i in reversed(value2):
                  
                    for j in i["values"]:
                     
                        if count2!=0:   
                            if j["field"] =="periodenddate":
                                end_date.append(Quarterlookup(DT.datetime.strptime(j['value'], '%m/%d/%Y')))
                       
                            ##Calculate change of revenue into the y_label 
                            if j["field"] =="totalrevenue":
                                revenue_cal =round(float(j['value'])/float(last_rev) - 1,3)
                                last_rev=j['value']
                                if revenue_cal>0:
                                    revenue_change.append(1)
                                else:
                                    revenue_change.append(0)
                        else:
                            ##Storing the initial quarter of revenue for calculation
                            if j["field"] =="totalrevenue":      
                                last_rev=j['value']
                              
                    count2+=1
            count1+=1
            
    ##Output will be a dictionary with stockname
    Outputy[stockname]["period"]=end_date
    Outputy[stockname]["revenuechange"]=revenue_change

In [12]:
##Writing y_tag to file
with open('Outputy.txt', 'w') as f:
    print(Outputy, file=f)

In [13]:
##Creating the train/Dev/test var
train_x_init = []
train_n_init = []
train_y_init = []
dev_x_init = []
dev_n_init = []
dev_y_init = []
test_x_init = []
test_n_init = []
test_y_init = []


In [14]:
##Putting the earlier dictinoary into the train var in order of quarter
trainquarter=['15Q1','15Q2','15Q3','15Q4','16Q1','16Q2','16Q3','16Q4','17Q1','17Q2','17Q3','17Q4','18Q1']

for i in trainstock:
    for j in range(len(trainquarter)):
        try:
                train_x_init.append(InputX [i][trainquarter[j]])
                train_n_init.append(InputN [i][trainquarter[j]])
                length=len(InputN [i][trainquarter[j]])           
                train_y_init.append([Outputy [i]['revenuechange'][j]]*length)
      
        except:
            pass



In [15]:
##Putting the earlier dictinoary into the dev/test var in order of quarter

testquarter=['18Q2','18Q3']

for i in teststock:
    for j in range(len(testquarter)):
        try:
            if testquarter[j]=='18Q3':
                test_x_init.append(InputX [i][testquarter[j]])
                test_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                test_y_init.append([Outputy [i]['revenuechange'][j]]*length)   
            
            elif testquarter[j]=='18Q2':
                dev_x_init.append(InputX [i][testquarter[j]])
                dev_n_init.append(InputN [i][testquarter[j]])
                length=len(InputN [i][testquarter[j]])
                dev_y_init.append([Outputy [i]['revenuechange'][j]]*length)     
               
      
        except:
            pass

In [16]:
##Creating the refined train/Dev/test var by flattening the shape of the original var
train_x_init2 = []
train_n_init2 = []
train_y_init2 = []
dev_x_init2 = []
dev_n_init2 = []
dev_y_init2 = []
test_x_init2 = []
test_n_init2 = []
test_y_init2 = []

for i in range(len(train_x_init)):
    for j in range(len(train_x_init[i])):
        train_x_init2.append(train_x_init[i][j])
        train_n_init2.append(train_n_init[i][j])
        train_y_init2.append(train_y_init[i][j])
        
for i in range(len(dev_x_init)):
    for j in range(len(dev_x_init[i])):
        dev_x_init2.append(dev_x_init[i][j])
        dev_n_init2.append(dev_n_init[i][j])
        dev_y_init2.append(dev_y_init[i][j])
        
for i in range(len(test_x_init)):
    for j in range(len(test_x_init[i])):
        test_x_init2.append(test_x_init[i][j])
        test_n_init2.append(test_n_init[i][j])
        test_y_init2.append(test_y_init[i][j])



# NEURAL BAG OF WORDS MODEL #

In [17]:
##Padding the refined var
def pad_np_array2(example_ids, max_len=50, pad_id=0):

    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr 

In [18]:
##Creating the final refined train/Dev/test var by padding it
train_x = pad_np_array2(train_x_init2, max_len = 50, pad_id=0)
train_n = np.array(train_n_init2)
train_y = np.array(train_y_init2)

test_x = pad_np_array2(test_x_init2, max_len = 50, pad_id=0)
test_n = np.array(test_n_init2)
test_y = np.array(test_y_init2)

dev_x = pad_np_array2(dev_x_init2, max_len = 50, pad_id=0)
dev_n = np.array(dev_n_init2)
dev_y = np.array(dev_y_init2)

In [19]:
##Setting up the NBoW Model
reload(models)

x, ns, y = train_x, train_n, train_y
batch_size = 125

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)
model_fn = models.classifier_model_fn

total_batches = 0
total_examples = 0
total_loss = 0
loss_ema = np.log(2)  # track exponential-moving-average of loss
ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
with tf.Graph().as_default(), tf.Session() as sess:
    ##
    # Construct the graph here. No session.run calls - just wiring up Tensors.
    ##
    # Add placeholders so we can feed in data.
    x_ph_  = tf.placeholder(tf.int32, shape=[None, x.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    ##
    # Done constructing the graph, now we can make session.run calls.
    ##
    sess.run(tf.global_variables_initializer())
    
    # Run a single epoch
    t0 = time.time()
    for (bx, bns, by) in utils.multi_batch_generator(batch_size, x, ns, y):
        # feed NumPy arrays into the placeholder Tensors
        feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
        batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
        # Compute some statistics
        total_batches += 1
        total_examples += len(bx)
        total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
        # Compute moving average to smooth out noisy per-batch loss
        loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
        if (total_batches % 125 == 0):
            print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
    print("Completed one epoch in {:s}".format(utils.pretty_timedelta(since=t0)))

15,625 examples, moving-average loss 0.34
31,250 examples, moving-average loss 0.91
46,875 examples, moving-average loss 1.01
62,500 examples, moving-average loss 0.23
78,125 examples, moving-average loss 0.28
93,750 examples, moving-average loss 0.26
109,375 examples, moving-average loss 0.22
125,000 examples, moving-average loss 0.32
140,625 examples, moving-average loss 0.27
156,250 examples, moving-average loss 0.25
171,875 examples, moving-average loss 0.24
187,500 examples, moving-average loss 0.35
203,125 examples, moving-average loss 0.31
218,750 examples, moving-average loss 0.25
234,375 examples, moving-average loss 0.51
250,000 examples, moving-average loss 0.27
265,625 examples, moving-average loss 0.72
281,250 examples, moving-average loss 0.27
296,875 examples, moving-average loss 0.26
312,500 examples, moving-average loss 0.27
328,125 examples, moving-average loss 0.28
343,750 examples, moving-average loss 0.29
359,375 examples, moving-average loss 0.35
375,000 examples,

2,906,250 examples, moving-average loss 0.26
2,921,875 examples, moving-average loss 0.50
2,937,500 examples, moving-average loss 0.29
2,953,125 examples, moving-average loss 0.28
2,968,750 examples, moving-average loss 0.47
2,984,375 examples, moving-average loss 0.28
3,000,000 examples, moving-average loss 0.27
3,015,625 examples, moving-average loss 0.47
3,031,250 examples, moving-average loss 0.29
3,046,875 examples, moving-average loss 0.27
3,062,500 examples, moving-average loss 0.46
3,078,125 examples, moving-average loss 0.28
3,093,750 examples, moving-average loss 0.27
3,109,375 examples, moving-average loss 0.46
3,125,000 examples, moving-average loss 0.28
3,140,625 examples, moving-average loss 0.27
3,156,250 examples, moving-average loss 0.49
3,171,875 examples, moving-average loss 0.28
3,187,500 examples, moving-average loss 0.27
3,203,125 examples, moving-average loss 0.35
3,218,750 examples, moving-average loss 0.28
3,234,375 examples, moving-average loss 0.27
3,250,000 

5,765,625 examples, moving-average loss 0.33
5,781,250 examples, moving-average loss 0.28
5,796,875 examples, moving-average loss 0.27
5,812,500 examples, moving-average loss 0.26
5,828,125 examples, moving-average loss 0.39
5,843,750 examples, moving-average loss 0.89
5,859,375 examples, moving-average loss 0.55
5,875,000 examples, moving-average loss 0.46
5,890,625 examples, moving-average loss 0.41
5,906,250 examples, moving-average loss 0.32
5,921,875 examples, moving-average loss 0.57
5,937,500 examples, moving-average loss 0.49
5,953,125 examples, moving-average loss 0.38
5,968,750 examples, moving-average loss 0.56
5,984,375 examples, moving-average loss 0.59
6,000,000 examples, moving-average loss 0.43
6,015,625 examples, moving-average loss 0.57
6,031,250 examples, moving-average loss 0.45
6,046,875 examples, moving-average loss 0.46
6,062,500 examples, moving-average loss 0.60
6,078,125 examples, moving-average loss 0.46
6,093,750 examples, moving-average loss 0.50
6,109,375 

In [20]:
##Setting up the NBoW Model
import models; reload(models)

# Specify model hyperparameters as used by model_fn
model_params = dict(V=vocab.size, embed_dim=25, hidden_dims=[20,20], num_classes=2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.1)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
# Write vocabulary to file, so TensorBoard can label embeddings.
# creates checkpoint_dir/projector_config.pbtxt and checkpoint_dir/metadata.tsv
vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (5,855 words) written to '/tmp/tf_bow_sst_20181218-0557/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20181218-0557/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20181218-0557', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc536db10b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20181218-0557' --port 6006

Then in your browser, open: http:

In [21]:
##Training the NBoW Model
# Training params, just used in this cell for the input_fn-s
train_params = dict(batch_size=125, total_epochs=10, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.
# Input function for training set batches
# Do 'eval_every' epochs at once, followed by evaluating on the dev set.
# NOTE: use patch_numpy_io.numpy_input_fn instead of tf.estimator.inputs.numpy_input_fn
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_n}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_n}, y=dev_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tf_bow_sst_20181218-0557/model.ckpt.
INFO:tensorflow:loss = 4.232431, step = 1
INFO:tensorflow:global_step/sec: 180.238
INFO:tensorflow:loss = 0.2732107, step = 101 (0.557 sec)
INFO:tensorflow:global_step/sec: 218.873
INFO:tensorflow:loss = 1.1208016, step = 201 (0.457 sec)
INFO:tensorflow:global_step/sec: 225.017
INFO:tensorflow:loss = 0.22146536, step = 301 (0.445 sec)
INFO:tensorflow:global_step/sec: 221.302
INFO:tensorflow:loss = 0.21379074, step = 401 (0.452 sec)
INFO:tensorflow:global_step/sec: 226.442
INFO:tensorflow:loss = 0.21087894, step = 501 (0.441 sec)
INFO:tensorflow:global_step/sec: 206.196
INFO:tensorflow:loss = 0.20940915, step = 601 (0.486 sec)
INFO:tensorflow:global_step/sec: 218.02
I

INFO:tensorflow:global_step/sec: 213.99
INFO:tensorflow:loss = 0.24915117, step = 8001 (0.467 sec)
INFO:tensorflow:global_step/sec: 221.652
INFO:tensorflow:loss = 0.24098817, step = 8101 (0.453 sec)
INFO:tensorflow:global_step/sec: 225.721
INFO:tensorflow:loss = 0.25397614, step = 8201 (0.442 sec)
INFO:tensorflow:global_step/sec: 224.25
INFO:tensorflow:loss = 0.23111525, step = 8301 (0.446 sec)
INFO:tensorflow:global_step/sec: 228.273
INFO:tensorflow:loss = 0.44948146, step = 8401 (0.438 sec)
INFO:tensorflow:global_step/sec: 222.123
INFO:tensorflow:loss = 0.29044914, step = 8501 (0.450 sec)
INFO:tensorflow:global_step/sec: 230.032
INFO:tensorflow:loss = 0.27078193, step = 8601 (0.434 sec)
INFO:tensorflow:global_step/sec: 234.832
INFO:tensorflow:loss = 0.25830993, step = 8701 (0.426 sec)
INFO:tensorflow:global_step/sec: 229.894
INFO:tensorflow:loss = 0.2967236, step = 8801 (0.441 sec)
INFO:tensorflow:global_step/sec: 221.519
INFO:tensorflow:loss = 0.26827568, step = 8901 (0.446 sec)
INF

INFO:tensorflow:global_step/sec: 217.426
INFO:tensorflow:loss = 0.71933794, step = 16201 (0.460 sec)
INFO:tensorflow:global_step/sec: 223.054
INFO:tensorflow:loss = 0.6673282, step = 16301 (0.448 sec)
INFO:tensorflow:global_step/sec: 214.721
INFO:tensorflow:loss = 0.5019628, step = 16401 (0.466 sec)
INFO:tensorflow:global_step/sec: 214.465
INFO:tensorflow:loss = 1.0336429, step = 16501 (0.466 sec)
INFO:tensorflow:global_step/sec: 218.906
INFO:tensorflow:loss = 0.7414031, step = 16601 (0.457 sec)
INFO:tensorflow:global_step/sec: 210.969
INFO:tensorflow:loss = 0.47717094, step = 16701 (0.478 sec)
INFO:tensorflow:global_step/sec: 207.832
INFO:tensorflow:loss = 0.7245724, step = 16801 (0.477 sec)
INFO:tensorflow:global_step/sec: 222.374
INFO:tensorflow:loss = 0.6891925, step = 16901 (0.451 sec)
INFO:tensorflow:global_step/sec: 221.283
INFO:tensorflow:loss = 0.47179782, step = 17001 (0.451 sec)
INFO:tensorflow:global_step/sec: 214.658
INFO:tensorflow:loss = 0.29254663, step = 17101 (0.466 s

INFO:tensorflow:global_step/sec: 149.638
INFO:tensorflow:loss = 0.26274186, step = 24401 (0.669 sec)
INFO:tensorflow:global_step/sec: 131.361
INFO:tensorflow:loss = 0.25455204, step = 24501 (0.764 sec)
INFO:tensorflow:global_step/sec: 159.622
INFO:tensorflow:loss = 0.4361534, step = 24601 (0.623 sec)
INFO:tensorflow:global_step/sec: 184.029
INFO:tensorflow:loss = 0.3314066, step = 24701 (0.543 sec)
INFO:tensorflow:global_step/sec: 178.5
INFO:tensorflow:loss = 0.30197617, step = 24801 (0.560 sec)
INFO:tensorflow:global_step/sec: 180.771
INFO:tensorflow:loss = 0.28511482, step = 24901 (0.553 sec)
INFO:tensorflow:global_step/sec: 182.408
INFO:tensorflow:loss = 0.35426357, step = 25001 (0.548 sec)
INFO:tensorflow:global_step/sec: 184.43
INFO:tensorflow:loss = 0.29741833, step = 25101 (0.542 sec)
INFO:tensorflow:global_step/sec: 176.672
INFO:tensorflow:loss = 0.27813563, step = 25201 (0.566 sec)
INFO:tensorflow:global_step/sec: 223.643
INFO:tensorflow:loss = 0.26611432, step = 25301 (0.447 

INFO:tensorflow:global_step/sec: 218.937
INFO:tensorflow:loss = 0.684789, step = 32601 (0.458 sec)
INFO:tensorflow:global_step/sec: 218.586
INFO:tensorflow:loss = 0.29466918, step = 32701 (0.457 sec)
INFO:tensorflow:global_step/sec: 218.997
INFO:tensorflow:loss = 0.58365554, step = 32801 (0.456 sec)
INFO:tensorflow:global_step/sec: 219.292
INFO:tensorflow:loss = 0.27577952, step = 32901 (0.456 sec)
INFO:tensorflow:global_step/sec: 214.019
INFO:tensorflow:loss = 0.70012844, step = 33001 (0.467 sec)
INFO:tensorflow:global_step/sec: 212.29
INFO:tensorflow:loss = 0.29133004, step = 33101 (0.471 sec)
INFO:tensorflow:global_step/sec: 210.067
INFO:tensorflow:loss = 0.6988287, step = 33201 (0.477 sec)
INFO:tensorflow:global_step/sec: 219.001
INFO:tensorflow:loss = 0.28281167, step = 33301 (0.456 sec)
INFO:tensorflow:global_step/sec: 224.191
INFO:tensorflow:loss = 0.7792421, step = 33401 (0.446 sec)
INFO:tensorflow:global_step/sec: 220.564
INFO:tensorflow:loss = 0.29170474, step = 33501 (0.453 

...

INFO:tensorflow:global_step/sec: 224.009
INFO:tensorflow:loss = 0.7588751, step = 498633 (0.446 sec)
INFO:tensorflow:global_step/sec: 221.598
INFO:tensorflow:loss = 0.67721707, step = 498733 (0.451 sec)
INFO:tensorflow:global_step/sec: 226.891
INFO:tensorflow:loss = 0.6305476, step = 498833 (0.441 sec)
INFO:tensorflow:global_step/sec: 226.856
INFO:tensorflow:loss = 0.6088929, step = 498933 (0.441 sec)
INFO:tensorflow:global_step/sec: 220.264
INFO:tensorflow:loss = 0.58849794, step = 499033 (0.454 sec)
INFO:tensorflow:global_step/sec: 225.762
INFO:tensorflow:loss = 0.81498265, step = 499133 (0.442 sec)
INFO:tensorflow:global_step/sec: 228.347
INFO:tensorflow:loss = 0.7836266, step = 499233 (0.439 sec)
INFO:tensorflow:global_step/sec: 218.946
INFO:tensorflow:loss = 0.6891388, step = 499333 (0.457 sec)
INFO:tensorflow:global_step/sec: 227.88
INFO:tensorflow:loss = 0.616593, step = 499433 (0.439 sec)
INFO:tensorflow:global_step/sec: 211.065
INFO:tensorflow:loss = 0.5958328, step = 499533 (

INFO:tensorflow:global_step/sec: 205.64
INFO:tensorflow:loss = 0.77785826, step = 506733 (0.487 sec)
INFO:tensorflow:global_step/sec: 212.349
INFO:tensorflow:loss = 0.6660893, step = 506833 (0.471 sec)
INFO:tensorflow:global_step/sec: 217.785
INFO:tensorflow:loss = 0.6185352, step = 506933 (0.458 sec)
INFO:tensorflow:global_step/sec: 214.112
INFO:tensorflow:loss = 0.59818053, step = 507033 (0.467 sec)
INFO:tensorflow:global_step/sec: 211.437
INFO:tensorflow:loss = 0.5790339, step = 507133 (0.473 sec)
INFO:tensorflow:global_step/sec: 211.022
INFO:tensorflow:loss = 0.76132303, step = 507233 (0.474 sec)
INFO:tensorflow:global_step/sec: 208.185
INFO:tensorflow:loss = 0.8152512, step = 507333 (0.480 sec)
INFO:tensorflow:global_step/sec: 213.456
INFO:tensorflow:loss = 0.7832767, step = 507433 (0.469 sec)
INFO:tensorflow:global_step/sec: 218.172
INFO:tensorflow:loss = 0.75320864, step = 507533 (0.458 sec)
INFO:tensorflow:global_step/sec: 215.51
INFO:tensorflow:loss = 0.667834, step = 507633 (

INFO:tensorflow:loss = 0.58324987, step = 514833 (0.495 sec)
INFO:tensorflow:global_step/sec: 213.125
INFO:tensorflow:loss = 0.79171956, step = 514933 (0.469 sec)
INFO:tensorflow:global_step/sec: 208.405
INFO:tensorflow:loss = 0.75863004, step = 515033 (0.481 sec)
INFO:tensorflow:global_step/sec: 219.789
INFO:tensorflow:loss = 0.61680216, step = 515133 (0.454 sec)
INFO:tensorflow:global_step/sec: 210.361
INFO:tensorflow:loss = 0.595025, step = 515233 (0.475 sec)
INFO:tensorflow:global_step/sec: 219.748
INFO:tensorflow:loss = 0.78056693, step = 515333 (0.455 sec)
INFO:tensorflow:global_step/sec: 215.812
INFO:tensorflow:loss = 0.74880254, step = 515433 (0.464 sec)
INFO:tensorflow:global_step/sec: 212.516
INFO:tensorflow:loss = 0.6242954, step = 515533 (0.471 sec)
INFO:tensorflow:global_step/sec: 209.866
INFO:tensorflow:loss = 0.6024272, step = 515633 (0.477 sec)
INFO:tensorflow:global_step/sec: 210.148
INFO:tensorflow:loss = 0.5820376, step = 515733 (0.475 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:loss = 0.696296, step = 522933 (0.473 sec)
INFO:tensorflow:global_step/sec: 217.019
INFO:tensorflow:loss = 0.6690754, step = 523033 (0.461 sec)
INFO:tensorflow:global_step/sec: 209.364
INFO:tensorflow:loss = 0.7043559, step = 523133 (0.478 sec)
INFO:tensorflow:global_step/sec: 215.465
INFO:tensorflow:loss = 0.6991055, step = 523233 (0.463 sec)
INFO:tensorflow:global_step/sec: 222.93
INFO:tensorflow:loss = 0.67243373, step = 523333 (0.449 sec)
INFO:tensorflow:global_step/sec: 211.893
INFO:tensorflow:loss = 0.70435816, step = 523433 (0.472 sec)
INFO:tensorflow:global_step/sec: 217.68
INFO:tensorflow:loss = 0.7018586, step = 523533 (0.460 sec)
INFO:tensorflow:global_step/sec: 213.035
INFO:tensorflow:loss = 0.6768265, step = 523633 (0.469 sec)
INFO:tensorflow:global_step/sec: 205.577
INFO:tensorflow:loss = 0.7027317, step = 523733 (0.487 sec)
INFO:tensorflow:global_step/sec: 206.305
INFO:tensorflow:loss = 0.70264643, step = 523833 (0.484 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 164.431
INFO:tensorflow:loss = 0.6111903, step = 531133 (0.608 sec)
INFO:tensorflow:global_step/sec: 206.839
INFO:tensorflow:loss = 0.59093076, step = 531233 (0.484 sec)
INFO:tensorflow:global_step/sec: 210.387
INFO:tensorflow:loss = 0.57071394, step = 531333 (0.475 sec)
INFO:tensorflow:global_step/sec: 214.137
INFO:tensorflow:loss = 0.8172928, step = 531433 (0.467 sec)
INFO:tensorflow:global_step/sec: 219.185
INFO:tensorflow:loss = 0.7833383, step = 531533 (0.456 sec)
INFO:tensorflow:global_step/sec: 219.227
INFO:tensorflow:loss = 0.75349385, step = 531633 (0.456 sec)
INFO:tensorflow:global_step/sec: 218.669
INFO:tensorflow:loss = 0.7257835, step = 531733 (0.458 sec)
INFO:tensorflow:global_step/sec: 209.532
INFO:tensorflow:loss = 0.6992302, step = 531833 (0.485 sec)
INFO:tensorflow:global_step/sec: 214.22
INFO:tensorflow:loss = 0.7043536, step = 531933 (0.458 sec)
INFO:tensorflow:global_step/sec: 217.9
INFO:tensorflow:loss = 0.6777022, step = 532033 (0

In [22]:
##Calculating the Accuracy

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_n}, y=test_y,
                    batch_size=125, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn = test_input_fn)  

print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-18-06:40:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181218-0557/model.ckpt-535415
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-18-06:40:25
INFO:tensorflow:Saving dict for global step 535415: accuracy = 0.56340367, cross_entropy_loss = 0.6850867, global_step = 535415, loss = 0.6870713
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 535415: /tmp/tf_bow_sst_20181218-0557/model.ckpt-535415
Accuracy on test set: 56.34%


{'accuracy': 0.56340367,
 'cross_entropy_loss': 0.6850867,
 'loss': 0.6870713,
 'global_step': 535415}

In [23]:
##Calculating the Precision
from sklearn.metrics import average_precision_score
    
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
average_precision = average_precision_score(y_pred,test_y)

print('Average precision score: {0:0.2f}'.format(
      average_precision))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20181218-0557/model.ckpt-535415
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Average precision score: nan


/home/danlee85/anaconda3/lib/python3.6/site-packages/sklearn/metrics/ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [24]:
##Loss Analysis
counttest=[]

for i in range(len(test_y_init)):
    counttest.append(len(test_y_init[i]))
    
count=0
for i in range(len(counttest)):
 
    lbl, cnts = np.unique(test_y[count:count+counttest[i]], return_counts=True)
    lbl2, cnts2 = np.unique (y_pred[count:count+counttest[i]], return_counts=True)
    print(teststock[i])
    print(list(zip(lbl2, cnts2)))
    print(lbl[0], "test vs predict", round(sum(lbl2*cnts2)/sum(cnts2),0))
    count+=counttest[i]



CVSHealth
[(0, 3739)]
1 test vs predict 0.0
RiteAid
[(0, 16608)]
0 test vs predict 0.0
Conn's
[(0, 18197)]
1 test vs predict 0.0
Casey's
[(0, 10040)]
1 test vs predict 0.0
Kroger
[(0, 2894)]
0 test vs predict 0.0
WeisMarkets
[(0, 3021)]
1 test vs predict 0.0
HavertyFurniture
[(0, 15229)]
0 test vs predict 0.0
Kirkland's
[(0, 27376)]
0 test vs predict 0.0
Pier1Imports
[(0, 24126)]
0 test vs predict 0.0
BuildersFirstSource
[(0, 25104)]
1 test vs predict 0.0
HuttigBuildingProducts
[(0, 25574)]
1 test vs predict 0.0
LumberLiquidators
[(0, 25079)]
0 test vs predict 0.0
TransWorldEntertainment
[(0, 25165)]
0 test vs predict 0.0
MarineMax
[(0, 23810)]
1 test vs predict 0.0
Medifast
[(0, 20200)]
0 test vs predict 0.0
SallyBeauty
[(0, 22235)]
1 test vs predict 0.0
Sotheby's
[(0, 19294)]
1 test vs predict 0.0
StarGasPartners
[(0, 26282)]
0 test vs predict 0.0
SuburbanPropanePartners
[(0, 26331)]
0 test vs predict 0.0
TitanMachinery
[(0, 22934)]
0 test vs predict 0.0
TractorSupply
[(0, 24267)]
1 

# LSTM Model

In [25]:
import rnnlm; reload(rnnlm)
import rnnlm_test; reload(rnnlm_test)
from __future__ import absolute_import
import unittest
from IPython.display import display, HTML

In [26]:
# reshape input x and y for LSTM model

print(len(train_x))
print(len(train_y))

print(train_y[0])
print(train_x[0])

train_labels = []
for i in range(len(train_y)):
    train_labels.append([train_y[i]] * 50)
    
print(train_labels[:5])

test_labels = []
for i in range(len(test_y)):
    test_labels.append([test_y[i]] * 50)

6692652
6692652
0
[1536 5846 5089 3292 5337  779   12 1801 5564  371  984 3201 3556 3548
  169 1301   12  473  169  781 3566 1351  169  453 2325 1506 5164 2094
 3548  169 2331 5706 3626 2630 2954 2957 3332  169 2466  169   66  780
  169    0    0    0    0    0    0    0]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
reload(rnnlm)

TF_GRAPHDIR = "/tmp/w266/a3_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=5855, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)

In [28]:
reload(rnnlm); reload(rnnlm_test)
utils.run_tests(rnnlm_test, ["TestRNNLMCore", "TestRNNLMTrain", "TestRNNLMSampler"])

test_shapes_embed (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_output (rnnlm_test.TestRNNLMCore) ... FAIL
test_shapes_recurrent (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_train (rnnlm_test.TestRNNLMTrain) ... ok
test_shapes_sample (rnnlm_test.TestRNNLMSampler) ... ERROR

ERROR: test_shapes_sample (rnnlm_test.TestRNNLMSampler)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/danlee85/w266/project/rnnlm_test.py", line 65, in test_shapes_sample
    self.assertEqual(self.lm.pred_samples_.get_shape().as_list(),
AttributeError: 'NoneType' object has no attribute 'get_shape'

FAIL: test_shapes_output (rnnlm_test.TestRNNLMCore)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/danlee85/w266/project/rnnlm_test.py", line 39, in test_shapes_output
    self.assertEqual(self.lm.W_out_.get_shape().as_list(), [100, 512])
AssertionError: Lists differ: [100, 2

In [29]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        feed_dict = {
            lm.input_w_: w,
            lm.target_y_: y,
            lm.initial_h_: h,
            lm.learning_rate_: learning_rate,
            lm.use_dropout_: use_dropout
        }
        ops = [loss, lm.final_h_, train_op]        
        #### YOUR CODE HERE ####
        # session.run(...) the ops with the feed_dict constructed above.
        # Ensure "cost" becomes the value of "loss".
        # Hint: see "ops" for other variables that need updating in this loop.
        

        cost, h, none = session.run(ops, feed_dict = feed_dict)
        
        
    
        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [30]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.rnnlm_batch_generator(train_ids, train_labels, batch_size=250)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=0.01, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

In [31]:
# Load the dataset
V = 5855
vocab, train_ids, test_ids = vocab, train_x, test_x

In [32]:
# Training parameters
max_time = 40
batch_size = 150
learning_rate = 0.01
num_epochs = 1

# Model parameters
model_params = dict(V=vocab.size, 
                    H=200, 
                    softmax_ns=200,
                    num_layers=2)

TF_SAVEDIR = "/tmp/w266/project"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [33]:
# Will print status every this many seconds
print_interval = 5

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.rnnlm_batch_generator(train_ids, train_labels, batch_size)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.

        run_epoch(lm, session, bi, train=True, verbose=True,tick_s=10, learning_rate=learning_rate)
        
        
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        
#         print("[epoch {:d}]".format(epoch), end=" ")
#         score_dataset(lm, session, train_ids, name="Train set")
#         print("[epoch {:d}]".format(epoch), end=" ")
#         score_dataset(lm, session, test_ids, name="Test set")
#         print("")
    
    # Save final model
    saver.save(session, trained_filename)

[epoch 1] Starting epoch 1
[batch 13]: seen 105000 words at 10432.3 wps, loss = 0.060
[batch 29]: seen 225000 words at 11046.7 wps, loss = 7.370
[batch 45]: seen 345000 words at 11200.0 wps, loss = 5.158
[batch 61]: seen 465000 words at 11274.6 wps, loss = 3.827
[batch 77]: seen 585000 words at 11318.8 wps, loss = 3.042
[batch 93]: seen 705000 words at 11338.8 wps, loss = 2.524
[batch 109]: seen 825000 words at 11354.8 wps, loss = 2.157
[batch 125]: seen 945000 words at 11370.4 wps, loss = 1.883
[batch 141]: seen 1065000 words at 11382.5 wps, loss = 1.671
[batch 157]: seen 1185000 words at 11392.2 wps, loss = 1.502
[batch 173]: seen 1305000 words at 11397.3 wps, loss = 1.364
[batch 189]: seen 1425000 words at 11397.6 wps, loss = 1.249
[batch 205]: seen 1545000 words at 11400.0 wps, loss = 1.314
[batch 221]: seen 1665000 words at 11400.1 wps, loss = 1.379
[batch 236]: seen 1777500 words at 11388.3 wps, loss = 1.683
[batch 252]: seen 1897500 words at 11387.5 wps, loss = 1.659
[batch 268]

[batch 2115]: seen 15870000 words at 11398.6 wps, loss = 0.875
[batch 2131]: seen 15990000 words at 11399.0 wps, loss = 0.893
[batch 2147]: seen 16110000 words at 11399.1 wps, loss = 0.886
[batch 2163]: seen 16230000 words at 11399.7 wps, loss = 0.880
[batch 2179]: seen 16350000 words at 11400.3 wps, loss = 0.873
[batch 2194]: seen 16462500 words at 11396.7 wps, loss = 0.867
[batch 2210]: seen 16582500 words at 11398.8 wps, loss = 0.861
[batch 2226]: seen 16702500 words at 11400.7 wps, loss = 0.855
[batch 2242]: seen 16822500 words at 11401.9 wps, loss = 0.863
[batch 2258]: seen 16942500 words at 11403.3 wps, loss = 0.872
[batch 2274]: seen 17062500 words at 11404.4 wps, loss = 0.866
[batch 2290]: seen 17182500 words at 11405.4 wps, loss = 0.860
[batch 2306]: seen 17302500 words at 11406.8 wps, loss = 0.854
[batch 2322]: seen 17422500 words at 11407.9 wps, loss = 0.848
[batch 2338]: seen 17542500 words at 11409.0 wps, loss = 0.842
[batch 2354]: seen 17662500 words at 11409.8 wps, loss 

[batch 4194]: seen 31462500 words at 11434.3 wps, loss = 0.723
[batch 4210]: seen 31582500 words at 11434.9 wps, loss = 0.722
[batch 4225]: seen 31695000 words at 11434.1 wps, loss = 0.729
[batch 4240]: seen 31807500 words at 11431.0 wps, loss = 0.728
[batch 4256]: seen 31927500 words at 11431.6 wps, loss = 0.732
[batch 4271]: seen 32040000 words at 11429.6 wps, loss = 0.732
[batch 4287]: seen 32160000 words at 11429.5 wps, loss = 0.733
[batch 4303]: seen 32280000 words at 11430.2 wps, loss = 0.735
[batch 4319]: seen 32400000 words at 11430.7 wps, loss = 0.734
[batch 4335]: seen 32520000 words at 11431.3 wps, loss = 0.737
[batch 4351]: seen 32640000 words at 11431.3 wps, loss = 0.736
[batch 4367]: seen 32760000 words at 11431.5 wps, loss = 0.739
[batch 4383]: seen 32880000 words at 11431.6 wps, loss = 0.739
[batch 4399]: seen 33000000 words at 11432.3 wps, loss = 0.738
[batch 4415]: seen 33120000 words at 11433.1 wps, loss = 0.736
[batch 4431]: seen 33240000 words at 11433.9 wps, loss 

[batch 6272]: seen 47047500 words at 11457.8 wps, loss = 0.643
[batch 6288]: seen 47167500 words at 11458.5 wps, loss = 0.642
[batch 6304]: seen 47287500 words at 11459.1 wps, loss = 0.640
[batch 6320]: seen 47407500 words at 11459.8 wps, loss = 0.639
[batch 6336]: seen 47527500 words at 11460.4 wps, loss = 0.637
[batch 6352]: seen 47647500 words at 11461.1 wps, loss = 0.636
[batch 6368]: seen 47767500 words at 11461.8 wps, loss = 0.644
[batch 6384]: seen 47887500 words at 11462.6 wps, loss = 0.647
[batch 6400]: seen 48007500 words at 11463.3 wps, loss = 0.646
[batch 6416]: seen 48127500 words at 11463.0 wps, loss = 0.644
[batch 6432]: seen 48247500 words at 11463.9 wps, loss = 0.643
[batch 6448]: seen 48367500 words at 11465.0 wps, loss = 0.641
[batch 6464]: seen 48487500 words at 11465.7 wps, loss = 0.640
[batch 6480]: seen 48607500 words at 11466.5 wps, loss = 0.639
[batch 6496]: seen 48727500 words at 11467.0 wps, loss = 0.637
[batch 6512]: seen 48847500 words at 11467.5 wps, loss 

[batch 8355]: seen 62670000 words at 11470.6 wps, loss = 0.551
[batch 8371]: seen 62790000 words at 11470.7 wps, loss = 0.550
[batch 8387]: seen 62910000 words at 11470.7 wps, loss = 0.578
[batch 8403]: seen 63030000 words at 11470.9 wps, loss = 0.577
[batch 8419]: seen 63150000 words at 11470.9 wps, loss = 0.575
[batch 8435]: seen 63270000 words at 11471.4 wps, loss = 0.574
[batch 8451]: seen 63390000 words at 11471.6 wps, loss = 0.573
[batch 8467]: seen 63510000 words at 11471.8 wps, loss = 0.572
[batch 8483]: seen 63630000 words at 11472.1 wps, loss = 0.571
[batch 8499]: seen 63750000 words at 11472.6 wps, loss = 0.570
[batch 8515]: seen 63870000 words at 11472.9 wps, loss = 0.569
[batch 8531]: seen 63990000 words at 11473.2 wps, loss = 0.568
[batch 8547]: seen 64110000 words at 11473.6 wps, loss = 0.583
[batch 8563]: seen 64230000 words at 11473.2 wps, loss = 0.582
[batch 8578]: seen 64342500 words at 11472.4 wps, loss = 0.581
[batch 8594]: seen 64462500 words at 11472.8 wps, loss 

...

[batch 34733]: seen 260505000 words at 11555.5 wps, loss = 0.496
[batch 34749]: seen 260625000 words at 11555.6 wps, loss = 0.496
[batch 34765]: seen 260745000 words at 11555.6 wps, loss = 0.501
[batch 34780]: seen 260857500 words at 11555.3 wps, loss = 0.503
[batch 34796]: seen 260977500 words at 11555.4 wps, loss = 0.502
[batch 34812]: seen 261097500 words at 11555.4 wps, loss = 0.502
[batch 34828]: seen 261217500 words at 11555.5 wps, loss = 0.502
[batch 34844]: seen 261337500 words at 11555.5 wps, loss = 0.502
[batch 34860]: seen 261457500 words at 11555.6 wps, loss = 0.501
[batch 34876]: seen 261577500 words at 11555.6 wps, loss = 0.501
[batch 34892]: seen 261697500 words at 11555.5 wps, loss = 0.501
[batch 34908]: seen 261817500 words at 11555.5 wps, loss = 0.501
[batch 34924]: seen 261937500 words at 11555.5 wps, loss = 0.505
[batch 34940]: seen 262057500 words at 11555.5 wps, loss = 0.506
[batch 34956]: seen 262177500 words at 11555.5 wps, loss = 0.505
[batch 34972]: seen 26229

[batch 36755]: seen 275670000 words at 11556.1 wps, loss = 0.511
[batch 36771]: seen 275790000 words at 11556.1 wps, loss = 0.511
[batch 36787]: seen 275910000 words at 11556.2 wps, loss = 0.513
[batch 36803]: seen 276030000 words at 11556.3 wps, loss = 0.514
[batch 36819]: seen 276150000 words at 11556.3 wps, loss = 0.513
[batch 36835]: seen 276270000 words at 11556.4 wps, loss = 0.513
[batch 36851]: seen 276390000 words at 11556.4 wps, loss = 0.513
[batch 36867]: seen 276510000 words at 11556.4 wps, loss = 0.513
[batch 36883]: seen 276630000 words at 11556.4 wps, loss = 0.513
[batch 36899]: seen 276750000 words at 11556.4 wps, loss = 0.512
[batch 36914]: seen 276862500 words at 11555.9 wps, loss = 0.512
[batch 36930]: seen 276982500 words at 11555.9 wps, loss = 0.512
[batch 36946]: seen 277102500 words at 11555.9 wps, loss = 0.512
[batch 36962]: seen 277222500 words at 11556.0 wps, loss = 0.513
[batch 36978]: seen 277342500 words at 11556.0 wps, loss = 0.514
[batch 36994]: seen 27746

[batch 38779]: seen 290850000 words at 11557.1 wps, loss = 0.510
[batch 38795]: seen 290970000 words at 11557.1 wps, loss = 0.511
[batch 38811]: seen 291090000 words at 11557.1 wps, loss = 0.511
[batch 38826]: seen 291202500 words at 11556.8 wps, loss = 0.511
[batch 38842]: seen 291322500 words at 11556.8 wps, loss = 0.511
[batch 38858]: seen 291442500 words at 11556.8 wps, loss = 0.510
[batch 38874]: seen 291562500 words at 11556.9 wps, loss = 0.510
[batch 38890]: seen 291682500 words at 11557.0 wps, loss = 0.510
[batch 38906]: seen 291802500 words at 11557.0 wps, loss = 0.510
[batch 38922]: seen 291922500 words at 11557.1 wps, loss = 0.510
[batch 38938]: seen 292042500 words at 11557.1 wps, loss = 0.511
[batch 38954]: seen 292162500 words at 11557.2 wps, loss = 0.511
[batch 38970]: seen 292282500 words at 11557.2 wps, loss = 0.511
[batch 38986]: seen 292402500 words at 11557.3 wps, loss = 0.511
[batch 39002]: seen 292522500 words at 11557.4 wps, loss = 0.510
[batch 39018]: seen 29264

[batch 40802]: seen 306022500 words at 11559.9 wps, loss = 0.517
[batch 40818]: seen 306142500 words at 11559.9 wps, loss = 0.517
[batch 40834]: seen 306262500 words at 11560.0 wps, loss = 0.517
[batch 40850]: seen 306382500 words at 11560.0 wps, loss = 0.516
[batch 40866]: seen 306502500 words at 11560.0 wps, loss = 0.516
[batch 40882]: seen 306622500 words at 11560.0 wps, loss = 0.516
[batch 40898]: seen 306742500 words at 11560.1 wps, loss = 0.516
[batch 40914]: seen 306862500 words at 11560.1 wps, loss = 0.517
[batch 40929]: seen 306975000 words at 11559.9 wps, loss = 0.517
[batch 40945]: seen 307095000 words at 11560.0 wps, loss = 0.517
[batch 40961]: seen 307215000 words at 11560.0 wps, loss = 0.517
[batch 40977]: seen 307335000 words at 11560.1 wps, loss = 0.517
[batch 40993]: seen 307455000 words at 11560.1 wps, loss = 0.517
[batch 41009]: seen 307575000 words at 11560.2 wps, loss = 0.516
[batch 41025]: seen 307695000 words at 11560.3 wps, loss = 0.516
[batch 41041]: seen 30781

[batch 42823]: seen 321180000 words at 11561.1 wps, loss = 0.527
[batch 42839]: seen 321300000 words at 11561.1 wps, loss = 0.527
[batch 42855]: seen 321420000 words at 11561.1 wps, loss = 0.527
[batch 42871]: seen 321540000 words at 11561.1 wps, loss = 0.527
[batch 42887]: seen 321660000 words at 11561.1 wps, loss = 0.527
[batch 42903]: seen 321780000 words at 11561.1 wps, loss = 0.526
[batch 42919]: seen 321900000 words at 11561.1 wps, loss = 0.526
[batch 42935]: seen 322020000 words at 11561.1 wps, loss = 0.526
[batch 42951]: seen 322140000 words at 11561.2 wps, loss = 0.526
[batch 42967]: seen 322260000 words at 11561.2 wps, loss = 0.526
[batch 42983]: seen 322380000 words at 11561.2 wps, loss = 0.525
[batch 42999]: seen 322500000 words at 11561.2 wps, loss = 0.525
[batch 43015]: seen 322620000 words at 11561.2 wps, loss = 0.525
[batch 43031]: seen 322740000 words at 11561.2 wps, loss = 0.525
[batch 43047]: seen 322860000 words at 11561.3 wps, loss = 0.527
[batch 43063]: seen 32298

In [42]:
def sample_step(lm, session, input_w, initial_h):
    """Run a single RNN step and return sampled predictions.
  
    Args:
      lm : rnnlm.RNNLM
      session: tf.Session
      input_w : [batch_size] vector of indices
      initial_h : [batch_size, hidden_dims] initial state
    
    Returns:
      final_h : final hidden state, compatible with initial_h
      samples : [batch_size, 1] vector of indices
    """
    # Reshape input to column vector
    input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
    #### YOUR CODE HERE ####
    # Run sample ops

    initial_h = session.run(lm.initial_h_, {lm.input_w_: input_w})
    
    feed_dict = {
        lm.input_w_: input_w,
        lm.initial_h_: initial_h,
        }
    
    ops = [lm.final_h_, lm.pred_max_]

    final_h, samples = session.run(ops, feed_dict = feed_dict)

    

    #### END(YOUR CODE) ####
    # Note indexing here: 
    #   [batch_size, max_time, 1] -> [batch_size, 1]
    return final_h, samples[:,-1]

In [55]:
# Same as above, but as a batch
max_steps = 20
num_samples = 100
random_seed = 42

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(random_seed)
    
    # Load the trained model
    saver.restore(session, trained_filename)

    # Make initial state for a batch with batch_size = num_samples
    w = np.array([test_x[:num_samples]])
    h = session.run(lm.initial_h_, {lm.input_w_: w[:,0]})
    pred_y = 0
    # We'll take one step for each sequence on each iteration 
    print(w)
    for i in range(max_steps-2):
        h, y = sample_step(lm, session, w[0][:,i+1], h)
#         np.hstack(pred_y, y)
        pred_y = y
        print(w)
        print(pred_y)

#     # Print generated sentences
#     for row in pred_y:
#         for pred in row:
#             print(pred)
        
    ##Calculate Accuracy, precision, recall
    i = 0
    true_pos = 0
    false_pos = 0
    false_neg = 0
    total_pred = 0 

    for y_pred, y_true in zip(pred_y, test_y[:num_samples]):
        total_pred += 1
        if y_pred == 1:
            if y_true == 1:
                true_pos += 1
            else:
                false_pos += 1
        elif y_true == 1:
            false_neg += 1
        i += 1
    
    if total_pred != 0:
        acc = true_pos / total_pred
    else:
        acc = 0
    if true_pos + false_pos != 0:
        pre = true_pos / (true_pos + false_pos)
    else:
        pre = 0
    if (true_pos + false_neg) != 0:
        rec = true_pos / (true_pos + false_neg)
    else:
        rec = 0
    
    print("Accuracy on test set: {:.02%}".format(acc))
    print("Precision on test set: {:.02%}".format(pre))
    print("Recall on test set: {:.02%}".format(rec))

INFO:tensorflow:Restoring parameters from /tmp/w266/project/rnnlm_trained
[[[1971 5036 5578 ...  316 5851    0]
  [1536 5846 5089 ...    0    0    0]
  [4801 5164  169 ...  873  340  585]
  ...
  [1971 5036 5578 ...  316 5851    0]
  [1297 2329  169 ...  169   64 2338]
  [ 169 5764 3548 ...    0    0    0]]]
[[[1971 5036 5578 ...  316 5851    0]
  [1536 5846 5089 ...    0    0    0]
  [4801 5164  169 ...  873  340  585]
  ...
  [1971 5036 5578 ...  316 5851    0]
  [1297 2329  169 ...  169   64 2338]
  [ 169 5764 3548 ...    0    0    0]]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[[1971 5036 5578 ...  316 5851    0]
  [1536 5846 5089 ...    0    0    0]
  [4801 5164  169 ...  873  340  585]
  ...
  [1971 5036 5578 ...  316 5851    0]
  [1297 2329  169 ...  169   64 2338]
  [ 169 5764 3548 ...    0    0    0]]]
[0 0 0 0 0 0 0